## Spectral Mesh Saliency

The code will try to reproduce the paper `Ran Song et al, Mesh Saliency via Spectral Processing` as much as possible.

Partial of the code will not be exactly same as the paper, but does the almost the same function. Such as the *Gaussian smoothing* of the Mesh


Also I will skip QSlim mesh simplification operation
since the built-in QSlim does not work on Python and Unix really well

The mesh $\mathcal{M}$ has been simplified by QSlim ahead of time and save to `./data/simp.off` file

The original $\mathcal{M}$ has been stored in `./data/0.off` file

In [2]:
# Import the libs
import numpy as np
import networkx as nx
from scipy import linalg as LA
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree
import scipy.io as sio

In [3]:
def read_off(file):
    """
    Function reads '.off' file
    """
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, n_dontknow = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    verts = np.array(verts)
    faces = np.array(faces)
    return verts, faces

In [4]:
def write_off(filename, vertex, face):
    """
    Function write up vertex anf face info to '.off' file
    """
    with open(filename, 'a') as f:
        f.write('OFF\n');
        f.write('%d %d 0\n'%(vertex.shape[0], face.shape[0]))
        np.savetxt(f, vertex, fmt='%f %f %f')
        np.savetxt(f, face, fmt='3 %d %d %d')

In [5]:
def deg_cal(in_faces):
    """
    Based on the input face info, aligned the format to be a graph format
    and read out the degree vector
    """
    out_faces = np.concatenate([in_faces[:,:2], in_faces[:,1:3], in_faces[:,[0,2]]], axis=0)
    out_faces = np.concatenate([out_faces,out_faces[:,::-1]], axis=0)
    out_faces = np.unique(out_faces, axis=0)
    _, deg = np.unique(out_faces[:,0], return_counts=True)
    return out_faces, deg

In [6]:
def simplifiedMapping(simp_vertex, org_vertex):
    """
    This function is able to map the simplified vertex
    to the orginal vertex by using k-d tree
    k = 1, d = 3
    """
    kd_tree = KDTree(simp_vertex)
    _, idx = kd_tree.query(org_vertex, k = 1)
    return idx

In [110]:
def gaussian_smoothing_dynamic(vertex_info, adj_mat, alpha=1):
    """
    Gaussian mesh smoothing, the sigma value is totally based on the 1-ring neighbors
    `alpha` is denoted as `t` in the paper
    """
    new_ver = np.zeros(vertex_info.shape).astype(np.float64)
    for i in range(vertex_info.shape[0]):
        adj_set = np.argwhere(np.array(adj_mat[i,:]).reshape(-1) == 1)
        adj_coord = vertex_info[adj_set,:].reshape(-1,3)
        ver_i_coord = vertex_info[i,:].reshape(-1,3)
        weight_set = np.zeros(adj_set.shape[0]).astype(np.float) 
        sigma = np.sum(np.sqrt(np.sum((ver_i_coord - adj_coord)**2, axis = 1)))/adj_set.shape[0] * alpha
        for j in range(adj_set.shape[0]):
            d_temp = np.sum((ver_i_coord - adj_coord[j,:])**2).astype(np.float64)
            weight_set[j] = (1/np.sqrt(2*np.pi*sigma**2)) * np.exp(-d_temp / (2*(sigma**2)))
        weight_set = weight_set / np.sum(weight_set)
        new_ver[i,:] = np.dot(weight_set.T, adj_coord)
    return new_ver

In [111]:
def gaussian_smoothing(vertex_info, adj_mat, alpha=1):
    """
    Another version of gaussian mesh smoothing with a constant alpha vlaue as the sigma
    """
    new_ver = np.zeros(vertex_info.shape).astype(np.float64)
    for i in range(vertex_info.shape[0]):
        adj_set = np.argwhere(np.array(adj_mat[i,:]).reshape(-1) == 1)
        adj_coord = vertex_info[adj_set,:].reshape(-1,3)
        ver_i_coord = vertex_info[i,:].reshape(-1,3)
        weight_set = np.zeros(adj_set.shape[0]).astype(np.float64) 
        sigma = alpha
        for j in range(adj_set.shape[0]):
            d_temp = np.sum((ver_i_coord - adj_coord[j,:])**2).astype(np.float64)
            weight_set[j] = (1/np.sqrt(2*np.pi*sigma**2)) * np.exp(-d_temp / (2*(sigma**2)))
        weight_set = weight_set / np.sum(weight_set)
        new_ver[i,:] = np.dot(weight_set.T, adj_coord)
    return new_ver

In [9]:
def append_eigen(eigen, N):
    if (N % 2) == 1:
        out_eign = np.concatenate([np.repeat(eigen[0], N/2), eigen, np.repeat(eigen[-1], N/2)])
    else:
        raise("Please input an odd number")
    return out_eign

In [10]:
def Measure_adjMat(face):
    """
    Return the adjcency matrix based on the input face
    """
    graph_edg_idx, deg_info = deg_cal(face)
    G = nx.Graph()
    G.add_edges_from(graph_edg_idx)
    adj_mat = nx.adjacency_matrix(G,
                                  nodelist=range(G.number_of_nodes())).todense().astype(float)
    return adj_mat

In [22]:
def RoughMeasure_Saliency(vers, fas):
    """
    Compute the Saliency based on one scale of vertex info
    """
    adj_mat = Measure_adjMat(fas)
    adj_cp = adj_mat.copy()
    
    adj_idx = np.argwhere(adj_mat==1)
    for idx in adj_idx:
        i, j = idx
        normy = (vers[i,:] - verts[j,:]) ** 2
        factor = np.sum(normy)
        adj_cp[i, j] = 1 / factor
    
    # Compute the Laplacian matrix
    Lap_mat = np.diagflat(np.sum(adj_cp ,axis=1)) - adj_cp
    # This step will take  the most of time, compute in O(n^3)
    # Python numpy sometimes give totally different eign_vec as MATLAB
    eign_val, eigen_vec = LA.eigh(Lap_mat)
    Log_eigen = np.log(np.abs(eign_val))
    # N is defined by paper
    N = 9
    spec_irregu = np.abs(Log_eigen -
                         np.convolve(append_eigen(Log_eigen,N),
                                     np.ones((N,))/N, mode='valid'))
    Saliency_mat = np.multiply(np.dot(np.dot(eigen_vec,
                                             np.diagflat(np.exp(spec_irregu))),
                                      eigen_vec.T), adj_cp)
    Saliency_vec = Saliency_mat.sum(axis=1).reshape(-1)
    return Saliency_vec

### The main function of program

In [12]:
# Read the orginal mesh, which includes 15000 vertices
with open('./data/0.off','r') as f:
    ori_verts, ori_faces = read_off(f)

# Read the pre-procesed QSlim simplified mesh, which only includes 1502 vertices, around 5000 faces    
with open('./data/simp.off','r') as f:
    verts, faces = read_off(f)    

##### Get the two lists of Mesh $\mathcal{\hat{M}}$ and $k\mathcal{\hat{M}}$

In [130]:
s = 10
dymG_mesh_lst = []
nmG_mesh_lst = []
adj_matt = Measure_adjMat(faces)
nmG_mesh_lst.append(verts)
dymG_mesh_lst.append(verts)
for ii in range(s-1):
    nmG_mesh_lst.append(gaussian_smoothing(verts, adj_matt, alpha=(ii+1)*2))
    dymG_mesh_lst.append(gaussian_smoothing_dynamic(verts, adj_matt, alpha=(ii+1)*2))

In [14]:
# # test chunk
# ! rm -f ./tst/main_tst_gaus5.off
# write_off('./tst/main_tst_gaus5.off', nmG_mesh_lst[4], faces)

# ! rm -f ./tst/main_tst_dygaus5.off
# write_off('./tst/main_tst_dygaus5.off', dymG_mesh_lst[4], faces)

In [133]:
Sal_mat = np.zeros((s, verts.shape[0])).astype(np.float)
Sal_ori = np.zeros((s, ori_verts.shape[0])).astype(np.float)
for idx in range(s):
    if idx == 0:
        Sal_hat = RoughMeasure_Saliency(nmG_mesh_lst[idx], faces)
    else:    
        Sal_tmp = RoughMeasure_Saliency(nmG_mesh_lst[idx], faces)
        Sal_tmp_dym = RoughMeasure_Saliency(dymG_mesh_lst[idx], faces) 
        Sal_hat = np.abs(Sal_tmp_dym - Sal_tmp) # Get the S_hat_i
    Sal_mat[idx,:] = Sal_hat
    
    idx_mapping = simplifiedMapping(nmG_mesh_lst[idx], ori_verts)
    Sal_ori[idx,:] = Sal_hat[:,idx_mapping].reshape(-1)

Sal_out = np.log(np.sum(Sal_ori, axis=0))

sio.savemat('./tst/sal.mat',{'sal':Sal_out})

###### Alternative algorthim

In [147]:
Sal_mat = np.zeros((s, ori_verts.shape[0])).astype(np.float)
Sal_ori = np.zeros((s-1, ori_verts.shape[0])).astype(np.float)
for idx in range(s):
    if idx == 0:
        Sal_hat = RoughMeasure_Saliency(nmG_mesh_lst[idx], faces)
    else:    
        Sal_tmp_dym = RoughMeasure_Saliency(dymG_mesh_lst[idx], faces) 
        Sal_hat = Sal_tmp_dym # Get the S_hat_i
    
    idx_mapping = simplifiedMapping(nmG_mesh_lst[idx], ori_verts)
    Sal_mat[idx,:] = Sal_hat[:,idx_mapping].reshape(-1)
    
    if idx != 0:
        Sal_ori[idx-1,:] = np.abs(Sal_mat[idx,:] - Sal_mat[idx-1,:])
    
Sal_out = np.log(np.sum(Sal_ori, axis=0))
sio.savemat('./tst/sal2.mat',{'sal2':Sal_out})